(своя тема)
Это анализатор для трех сообществ вконтакте (новостное, футбольное и историческое). 
Для каждого сообщества программа находит самое популярное слово за последние 100 постов (без учета стоп-слов) и, используя его 
как начальный узел для семантического поля, строит граф. 

In [5]:
import urllib.request
import json
import re
import os
#from pymystem3 import Mystem
import Anaconda
import gensim
from gensim.models import word2vec
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import style
from networkx.algorithms import community

ModuleNotFoundError: No module named 'gensim'

In [1]:
def words():
    vk_ids = ['-29534144', '-217557', '-95380917']
    tops = ''
    
    for i in vk_ids:
        vk_id = i
        
        token = 'ce9bbc81ce9bbc81ce9bbc81f7cef1aec5cce9bce9bbc8192236d' \
                '27163fe4bf62eb1946'
        req = urllib.request.Request(
        'https://api.vk.com/method/wall.get?owner_id=%s&count=5&'
        'access_token=%s&v=5.95' % (vk_id, token))
        response = urllib.request.urlopen(req)
        result = response.read().decode('utf-8')
        data = json.loads(result)
        posts = data['response']['items']
        
        all_text = ''
        #m = Mystem()
        #sw = stopwords.words('russian')

        for post in posts:
            text = post['text']
            plain = re.sub('[^а-яА-Я\s]', '', text)
            all_text += plain
            
            
        m = Mystem()
        lemmas = m.lemmatize(str(all_text))
        #print(lemmas)
        
        top_lem = Counter(lemmas)
        top_lem = dict(top_lem)
        top_lem = {c: top_lem[c] for c in top_lem if top_lem[c] > 2 and len(c) > 2}
        top_lem = (sorted(top_lem.items(), key=lambda x: x[1], reverse=True))[:1]
        
        print(top_lem)
        
        for i in dict(top_lem).keys():
            tops += i + ' '
        
    with open('tops.txt', 'w+', 'encoding=utf-8') as f:
        f.write(tops)
            
        #new_lines = []
        #for line in all_text:
            #line = ' '.join([w for w in line.split() if w not in sw])
        #    newline = ''.join(m.lemmatize(line))
        #    new_lines.append(newline)
            
        #print(new_lines)

In [21]:
if __name__ == '__main__':
    words()

[('задерживать', 6)]
[]
[('тайна', 3)]
задерживатьтайна


In [22]:
urllib.request.urlretrieve(
    'http://rusvectores.org/static/models/rusvectores2/'
    'ruscorpora_mystem_cbow_300_2_2015.bin.gz',
    'ruscorpora_mystem_cbow_300_2_2015.bin.gz')

NameError: name 'tops' is not defined

In [ ]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

In [ ]:
with open('tops.txt', 'r', 'encoding=utf-8') as f:
    words = list(f.read())

In [ ]:
G = nx.Graph()

# список с соседями первого порядка
level_one = []

for word in words:
    if word in model:
        print(word)
        word_part = str(word)[-1:]
        print(word_part)
        #  количество слов-соседей для первого узла
        syn_words = len(model[word])
        for i in model.most_similar(positive=[word], topn=syn_words):
            if i[1] >= 0.5 and str(i[0])[-1:] == word_part:
                level_one.append(i[0])
                G.add_node(i[0], label=str(i[0]))
                G.add_edge(str(word), str(i[0]))
    else:
        print('Увы, слова "%s" нет в модели!' % word)